In [2]:
import numpy as np
import scipy as sc
import pandas as pd
import mangoes
import nltk
from gensim.models.word2vec import PathLineSentences

In [3]:
language = 'english'
path_data = './../semeval2020_ulscd_posteval/starting_kit/test_data_public'

In [4]:
corpus1_lemma = mangoes.Corpus(f'{path_data}/{language}/corpus1/lemma')
print('== Corpus 1 ==')
print(corpus1_lemma.nb_sentences,"sentences")
print(len(corpus1_lemma._words_count),"words")


corpus2_lemma = mangoes.Corpus(f'{path_data}/{language}/corpus2/lemma')
print('== Corpus 2 ==')
print(corpus2_lemma.nb_sentences,"sentences")
print(len(corpus2_lemma._words_count),"words")



Counting words: 0it [00:00, ?it/s]

== Corpus 1 ==
253644 sentences
86557 words


Counting words: 0it [00:00, ?it/s]

== Corpus 2 ==
353692 sentences
149891 words


In [54]:
list(set(vocab1.words) & set(vocab2.words))

33954

In [8]:
vocab1 = corpus1_lemma.create_vocabulary()
print(len(vocab1),"words")
vocab2 = corpus2_lemma.create_vocabulary()
print(len(vocab2),"words")
vocabulary = mangoes.Vocabulary(list(set(vocab1.words) & set(vocab2.words)))

counts_matrix1 = mangoes.counting.count_cooccurrence(corpus1_lemma, vocabulary, vocabulary)
counts_matrix2 = mangoes.counting.count_cooccurrence(corpus2_lemma, vocabulary, vocabulary)


86557 words
149891 words


In [9]:
ppmi1 = mangoes.create_representation(counts_matrix1, weighting=mangoes.weighting.PPMI()).to_df()
ppmi2 = mangoes.create_representation(counts_matrix2, weighting=mangoes.weighting.PPMI()).to_df()

In [ ]:
targets = mangoes.Corpus(f'{path_data}/{language}/targets')

In [11]:
sc.spatial.distance.cosine(ppmi1['he'],ppmi2['he'])

0.7775834667169774